#Simple ETL with Pandas
Extract, Transform dan Load (ETL) merupakan kumpulan proses untuk "memindahkan" data dari satu tempat ke tempat lain, yaitu dari sumber data (bisa berupa database aplikasi, file, logs, database dari 3rd party, dan lainnya) ke data warehouse.

In [10]:
#Extract
import pandas as pd
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')

print(df_participant.shape)
print(df_participant.info())
print(df_participant.head())

(5000, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   participant_id  5000 non-null   object 
 1   first_name      5000 non-null   object 
 2   last_name       5000 non-null   object 
 3   birth_date      5000 non-null   object 
 4   address         5000 non-null   object 
 5   phone_number    5000 non-null   object 
 6   country         5000 non-null   object 
 7   institute       5000 non-null   object 
 8   occupation      5000 non-null   object 
 9   register_time   5000 non-null   float64
dtypes: float64(1), object(9)
memory usage: 390.8+ KB
None
                         participant_id  ... register_time
0  bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a  ...  1.617634e+09
1  7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3  ...  1.617634e+09
2  19582d7f-b824-4fe5-a517-d5bf573fc768  ...  1.617634e+09
3  aeb6d817-25f3-4867-8a74-8d92e0a0f633  ...  1

#####Transform
* Merubah nilai dari suatu kolom ke nilai baru,
* Menciptakan kolom baru dengan memanfaatkan kolom lain,
* Transpose baris menjadi kolom (atau sebaliknya),
* Mengubah format data ke bentuk yang lebih standard (contohnya, kolom date maupun datetime yang biasanya memiliki nilai yang tidak standard maupun nomor * HP yang biasanya memiliki nilai yang tidak sesuai format standardnya), dan lainnya. 


In [13]:
print(df_participant['address'].head())

#Transform Bagian I - Kode Pos
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$') #Masukkan regex didalam fungsi extract
print(df_participant['postal_code'].head())

0             Gg. Monginsidi No. 08\nMedan, Aceh 80734
1       Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434
2               Jalan Kebonjati No. 0\nAmbon, SS 57739
3      Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156
4    Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...
Name: address, dtype: object
0    80734
1    09434
2    57739
3    76156
4    70074
Name: postal_code, dtype: object


In [14]:
#Transform Bagian II - Kota
#Masukkan regex didalam fungsi extract
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')
print(df_participant['city'].head())

0           Medan
1      Prabumulih
2           Ambon
3    Lubuklinggau
4          Kediri
Name: city, dtype: object


In [23]:
print(df_participant[['first_name', 'last_name']].head())
#Transform Bagian III - Github
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()
print(df_participant['github_profile'].head())

  first_name   last_name
0      Citra  Nurdiyanti
1       Aris    Setiawan
2   Cornelia   Handayani
3      Soleh      Rajasa
4       Vivi      Astuti
0      https://github.com/citranurdiyanti
1         https://github.com/arissetiawan
2    https://github.com/corneliahandayani
3          https://github.com/solehrajasa
4           https://github.com/viviastuti
Name: github_profile, dtype: object


In [31]:
#Transform Bagian IV - Nomor Handphone
#Jika awalan nomor HP berupa angka 62 atau +62 yang merupakan kode telepon Indonesia, maka diterjemahkan ke 0.
#Tidak ada tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-
#Tidak ada spasi pada nomor HP nama kolom untuk menyimpan hasil cleansing pada nomor HP yaitu cleaned_phone_number
print(df_participant['phone_number'].head())
#Masukkan regex pada parameter pertama dari fungsi replace
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')

print(df_participant['cleaned_phone_number'].head())

0       (0151) 081 2706
1    +62 (036) 461 7027
2          089 833 6695
3    +62 (418) 329-4756
4            0812511835
Name: phone_number, dtype: object
0    01510812706
1    00364617027
2     0898336695
3    04183294756
4     0812511835
Name: cleaned_phone_number, dtype: object


In [32]:
print(df_participant[['first_name','last_name','country','institute']].head())

#Transform Bagian V - Nama Tim
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) #Singkatan dari Nama Depan dan Nama Belakang dengan mengambil huruf pertama
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Singkatan dari value di kolom institute
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)
print(df_participant['team_name'].head())

  first_name   last_name      country                  institute
0      Citra  Nurdiyanti      Georgia       UD Prakasa Mandasari
1       Aris    Setiawan  Korea Utara     Universitas Diponegoro
2   Cornelia   Handayani       Komoro   UD Hardiansyah Puspasari
3      Soleh      Rajasa      Eritrea  Perum Tampubolon Yuliarti
4       Vivi      Astuti     Aljazair     PT Hardiansyah Rahimah
0       CN-Georgia-UPM
1    AS-Korea Utara-UD
2        CH-Komoro-UHP
3       SR-Eritrea-PTY
4      VA-Aljazair-PHR
Name: team_name, dtype: object


In [33]:
#Transform Bagian VI - Email
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)
print(df_participant['email'].head())

0      citranurdiyanti@upm.com
1        arissetiawan@ud.ac.ku
2    corneliahandayani@uhp.com
3          solehrajasa@pty.com
4           viviastuti@phr.com
Name: email, dtype: object


In [39]:
# Transform Bagian VII - Tanggal Lahir
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')
print(df_participant['birth_date'].head())

0   1991-02-05
1   1993-01-11
2   1993-07-31
3   1991-11-04
4   2003-01-22
Name: birth_date, dtype: datetime64[ns]


In [40]:
#Transform Bagian VII - Tanggal Daftar Kompetisi
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')
print(df_participant['register_time'].head())

0    1.617634e+09
1    1.617634e+09
2    1.617634e+09
3    1.617634e+09
4    1.617634e+09
Name: register_time, dtype: float64


In [41]:
print(df_participant.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   participant_id        5000 non-null   object        
 1   first_name            5000 non-null   object        
 2   last_name             5000 non-null   object        
 3   birth_date            5000 non-null   datetime64[ns]
 4   address               5000 non-null   object        
 5   phone_number          5000 non-null   object        
 6   country               5000 non-null   object        
 7   institute             5000 non-null   object        
 8   occupation            5000 non-null   object        
 9   register_time         5000 non-null   float64       
 10  postal_code           5000 non-null   object        
 11  city                  5000 non-null   object        
 12  github_profile        5000 non-null   object        
 13  cleaned_phone_numb

Dari data kolom sebelumnya saat proses extract, ada beberapa kolom tambahan yang didapatkan dengan memanfaatkan kolom. Saat ini dataset memuat 16 kolom.